In [57]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

TRADUCTOR_SIGFE_SIGCOM = pd.read_excel('input\\asociacion_sigfe_sigcom.xlsx')
TRADUCTOR_SIGFE_SIGCOM['Cod. SIGFE limpio'] = TRADUCTOR_SIGFE_SIGCOM['Cod. SIGFE limpio'].str.replace('.', '', regex = False)
TRADUCTOR_SIGFE_SIGCOM['Cod. SIGCOM'] = TRADUCTOR_SIGFE_SIGCOM['Cod. SIGCOM'].astype(int)
TRADUCTOR_SIGFE_SIGCOM = TRADUCTOR_SIGFE_SIGCOM.set_index('Cod. SIGFE limpio')

EXCEPCIONES_SIGFE = {
    '221299901601': 'FERNANDO BARAONA EN RRHH. ANEDIN EN GASTO GENERAL',
    '221299901602': 'Todo en RRHH',
    '221299900902': 'Cardiologia y Cardiocirugia en RRHH. UC Christus en gastos generales',
    '221299900201': 'M Meneses cargado en RRHH',
    '221299900202': 'J Andueza y Cardiocirugia en RRHH'
}

In [133]:
def desglosar_excepciones(agrupados_por_sigcom):
    df_devengos = pd.read_excel('input\\SA_DisponibilidadDevengoPresupuestario.xls', header = 5)
    df_devengos = df_devengos[['Titulo', 'Principal', 'Concepto Presupuestario', 'Monto Vigente', 'Monto Disponible', 'Monto Consumido']]
    df_devengos['Item Presupuestario'] = df_devengos['Concepto Presupuestario'].str.split() \
                                                                            .str[0]

    for concepto_pres_excepcion in EXCEPCIONES_SIGFE.keys():
        query_excepcion = df_devengos.query('`Item Presupuestario` == @concepto_pres_excepcion')

        if concepto_pres_excepcion == '221299901601':
            # FER BARAONA EN RRHH, todo lo demas en gastos generales
            mask_a_rrhh = query_excepcion['Principal'].str.contains('BARAONA')
        
        elif concepto_pres_excepcion == '221299901602':
            # Todo en RRHH
            mask_a_rrhh = query_excepcion['Principal'].notna()
        
        elif concepto_pres_excepcion == '221299900902':
            # CARDIOCIRUGUA Y CARDIOLOGIA EN RRHH, LO DEMAS EN GASTOS GENERALES.
            mask_a_rrhh = (query_excepcion['Principal'].str.contains('CARDIOLOGIA')) | \
                        (query_excepcion['Principal'].str.contains('CARDIOCIRUGIA'))

        elif concepto_pres_excepcion == '221299900201':
            # M MENESES EN RRHH, LO DEMAS EN GASTOS GENERALES
            mask_a_rrhh = query_excepcion['Principal'].str.contains('MANUEL MENESES')
            
        elif concepto_pres_excepcion == '221299900202':
            # ANDUEZA Y CARDIOCIRUGIA EN RRHH, LO DEMAS EN GASTOS GENERALES.
            mask_a_rrhh = query_excepcion['Principal'].str.contains('ANDUEZA') | \
                        query_excepcion['Principal'].str.contains('CARDIOCIRUGIA')
        
        df_a_rrhh = query_excepcion[mask_a_rrhh]
        df_a_gg = query_excepcion[~mask_a_rrhh]

        valor_a_rrhh = df_a_rrhh['Monto Vigente'].sum()
        valor_a_gg = df_a_gg['Monto Vigente'].sum()

        mask_excepcion = (agrupados_por_sigcom['Item Presupuestario'] == concepto_pres_excepcion)

        agrupados_por_sigcom.loc[mask_excepcion, 'Devengado_RRHH'] = valor_a_rrhh 
        agrupados_por_sigcom.loc[mask_excepcion, 'Devengado_GG'] = valor_a_gg
    
    return agrupados_por_sigcom

In [134]:
df = pd.read_excel('input\\SA_EstadoEjecucionPresupuestaria.xls', header = 6)
df['Item Presupuestario'] = df['Concepto Presupuestario'].str.split() \
                                                         .str[0]
df = df.set_index('Item Presupuestario')


In [135]:
columnas_utiles = ['Devengado']
agrupar_por = ['Cod. SIGCOM', 'Item en SIGCOM', 'Item Presupuestario']

unidas = pd.merge(df, TRADUCTOR_SIGFE_SIGCOM, how = 'left', left_index = True, right_index = True)
agrupados_por_sigcom = unidas.groupby(agrupar_por, dropna = False)[columnas_utiles].sum().reset_index()
agrupados_por_sigcom = desglosar_excepciones(agrupados_por_sigcom)

In [138]:
agrupados_por_sigcom.to_excel('Totales.xlsx', index = False)

PermissionError: [Errno 13] Permission denied: 'Totales.xlsx'

In [109]:
# Bien!
agrupados_por_sigcom.reset_index().query('`Item Presupuestario` == "221299901601"')

,Cod. SIGCOM,Item en SIGCOM,Item Presupuestario,Devengado
17,61.0,COMPRA DE CONSULTAS MÉDICAS *,221299901601,55730835


In [110]:
# Todo esto va a RRHH!
agrupados_por_sigcom.reset_index().query('`Item Presupuestario` == "221299901602"')

,Cod. SIGCOM,Item en SIGCOM,Item Presupuestario,Devengado
18,62.0,COMPRA DE CONSULTAS NO MÉDICAS *,221299901602,38233300


In [115]:

print(f'La cartola de ejecucion presupuestaria tiene un devengado de: 198335431\n'
      f'La cartola de devengados tiene un devengado de: 201784611 (y todo va a RRHH)')

print(f'Hay una diferencia de: {201784611 - 198335431}')
agrupados_por_sigcom.reset_index().query('`Item Presupuestario` == "221299900902"')

La cartola de ejecucion presupuestaria tiene un devengado de: 198335431
La cartola de devengados tiene un devengado de: 201784611 (y todo va a RRHH)
Hay una diferencia de: 3449180


,Cod. SIGCOM,Item en SIGCOM,Item Presupuestario,Devengado
233,NaN,NaN,221299900902,198335431


In [117]:
print(f'El devengado total es de: 10198578')
print(f'9220752 va a GG')
print(f'977826 va a RRHH')

print(f'{9220752 + 977826}')
agrupados_por_sigcom.reset_index().query('`Item Presupuestario` == "221299900201"')

El devengado total es de: 10198578
9220752 va a GG
977826 va a RRHH
10198578


,Cod. SIGCOM,Item en SIGCOM,Item Presupuestario,Devengado
44,177.0,SERVICIO DE LABORATORIO,221299900201,10198578


In [118]:
agrupados_por_sigcom.reset_index().query('`Item Presupuestario` == "221299900202"')
print(f'Si cuadran!')

,Cod. SIGCOM,Item en SIGCOM,Item Presupuestario,Devengado
232,NaN,NaN,221299900202,176642750
